In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [2]:
import random

def endgame(state):
    rows = 7
    cols = 6
    check = [1, 2] # players are represented by 1 and 2
    for p in check:
        # check for horizontal win
        for r in range(rows):
            for c in range(cols - 3):
                if state[r*cols+c] == p and state[r*cols+c+1] == p and state[r*cols+c+2] == p and state[r*cols+c+3] == p:
                    return p
        # check for vertical win
        for r in range(rows - 3):
            for c in range(cols):
                if state[r*cols+c] == p and state[(r+1)*cols+c] == p and state[(r+2)*cols+c] == p and state[(r+3)*cols+c] == p:
                    return p
        # check for diagonal win (positive slope)
        for r in range(rows - 3):
            for c in range(cols - 3):
                if state[r*cols+c] == p and state[(r+1)*cols+c+1] == p and state[(r+2)*cols+c+2] == p and state[(r+3)*cols+c+3] == p:
                    return p
        # check for diagonal win (negative slope)
        for r in range(3, rows):
            for c in range(cols - 3):
                if state[r*cols+c] == p and state[(r-1)*cols+c+1] == p and state[(r-2)*cols+c+2] == p and state[(r-3)*cols+c+3] == p:
                    return p
    # check if the board is full
    if all(x != 0 for x in state):
        return -1
    # if no winner and the board is not full, the game is still ongoing
    return 0


In [3]:
def find_mov(state, turn):
    side = 1 if turn == 1 else -1
    empty_pos = [i for i in range(len(state)) if state[i] == 0]
    if random.random() < epsilon:
        first_try = True
        for pos in empty_pos:
            if (state, pos) not in Q:
                Q[(state, pos)] = random.random() * 2 - 1
            if first_try:
                max_Q = side * Q[(state, pos)]
                max_a = pos
                first_try = False
            else:
                if side * Q[(state, pos)] > max_Q:
                    max_Q = side * Q[(state, pos)]
                    max_action = pos
        return pos
    return random.choice(empty_pos)

In [4]:
def reward(state):
    ret = endgame(state)
    if ret == 1:
        return 1000
    if ret == 2:
        return -1000
    return 0

In [5]:
def available_moves(board):
    moves = []
    rows = 7
    cols = 6
    for i in range(cols):
        for j in range(rows-1,-1,-1):
            if board[(j*cols) + i] == 0:
                moves.append((j*cols) + i)
                break
    return moves

In [6]:
def max_action(Q, state):
    state = tuple(state)
    empty_pos = available_moves(state)
    first_try = True
    max_action = -1
    max_Q = 0
    for pos in empty_pos:
        if (state, pos) not in Q:
#             print('sd')
            Q[(state, pos)] = random.random() * 2 - 1
        if first_try:
            max_Q = Q[(state, pos)]
            max_action = pos
            first_try = False
        else:
            if Q[(state, pos)] > max_Q:
                max_Q = Q[(state, pos)]
                max_action = pos
    return max_action, max_Q

In [7]:
def min_action(Q, state):
    state = tuple(state)
    empty_pos = available_moves(state)
    first_try = True
    min_a = -1
    min_Q = 0
    for pos in empty_pos:
        if (state, pos) not in Q:
            Q[(state, pos)] = random.random() * 2 - 1
        if first_try:
            min_Q = Q[(state, pos)]
            min_a = pos
            first_try = False
        else:
            if Q[(state, pos)] < min_Q:
                min_Q = Q[(state, pos)]
                min_a = pos
    return min_a, min_Q

In [8]:
def show_state(state):
    for i in range(3):
        for j in range(3):
            print(state[i * 3 + j], end=" ")
        print()
    print()

In [9]:
def QAgent(alpha, gamma, epsilon, itera):
    Q = dict()
    for i in range(itera):
        current_state = (0,0,0,0,0,0,
             0,0,0,0,0,0,
             0,0,0,0,0,0,
             0,0,0,0,0,0,
             0,0,0,0,0,0,
             0,0,0,0,0,0,
             0,0,0,0,0,0)

        while endgame(current_state) == 0:
            # O Moves (1's Turn)
            turn = 1
            next_state = list(current_state)
            mov = find_mov(current_state, turn)
            next_state[mov] = turn
            m_a, m_Q = min_action(Q, next_state)
            if (current_state, mov) not in Q:
                Q[(current_state, mov)] = random.random() * 2 - 1
            Q[(current_state, mov)] = Q[(current_state, mov)] + alpha * (
                reward(next_state) + gamma * m_Q - Q[(current_state, mov)]
            )
            current_state = tuple(next_state)
            if endgame(current_state) != 0:
                break
            current_state = tuple(next_state)

            # X Moves (2's Turn)
            turn = 2
            next_state = list(current_state)
            mov = find_mov(current_state, turn)
            next_state[mov] = turn
            m_a, m_Q = max_action(Q, next_state)
            if (current_state, mov) not in Q:
                Q[(current_state, mov)] = random.random() * 2 - 1
            Q[(current_state, mov)] = Q[(current_state, mov)] + alpha * (
                reward(next_state) + gamma * m_Q - Q[(current_state, mov)]
            )
            current_state = tuple(next_state)
            if endgame(current_state) != 0:
                break
            current_state = tuple(next_state)
    return Q

In [10]:
def printboard(state):
    state = [' ' if x==0 else x for x in state]
    state = ['x' if x==1 else x for x in state]
    state = ['o' if x==2 else x for x in state]
    print('| {} | {} | {} | {} | {} | {} |'.format(state[0], state[1], state[2], state[3], state[4], state[5]))
#     print('---+---+---+---+---+---')
    print('| {} | {} | {} | {} | {} | {} |'.format(state[6], state[7], state[8], state[9], state[10], state[11]))
#     print('---+---+---+---+---+---')
    print('| {} | {} | {} | {} | {} | {} |'.format(state[12], state[13], state[14], state[15], state[16], state[17]))
#     print('---+---+---+---+---+---')
    print('| {} | {} | {} | {} | {} | {} |'.format(state[18], state[19], state[20], state[21], state[22], state[23]))
#     print('---+---+---+---+---+---')
    print('| {} | {} | {} | {} | {} | {} |'.format(state[24], state[25], state[26], state[27], state[28], state[29]))
#     print('---+---+---+---+---+---')
    print('| {} | {} | {} | {} | {} | {} |'.format(state[30], state[31], state[32], state[33], state[34], state[35]))
    
    print('| {} | {} | {} | {} | {} | {} |'.format(state[36], state[37], state[38], state[39], state[40], state[41]))
    print('\n')


In [11]:
def check_potential_win(board, player):
    # Check horizontal potential wins
    for i in range(6):
        for j in range(3):
#             print(i*6+j)
            if board[i*6+j] == player and board[i*6+j+1] == player and board[i*6+j+2] == player and board[i*6+j+3] == 0:
#                 print('a')
                return j
            if board[i*6+j] == player and board[i*6+j+1] == player and board[i*6+j+3] == player and board[i*6+j+2] == 0:
#                 print('b')
                return j
            if board[i*6+j] == player and board[i*6+j+2] == player and board[i*6+j+3] == player and board[i*6+j+1] == 0:
#                 print('c')
                return j
            if board[i*6+j+1] == player and board[i*6+j+2] == player and board[i*6+j+3] == player and board[i*6+j] == 0:
#                 print('d')
                return j
    
    # Check vertical potential wins
    for j in range(6):
        for i in range(4):
#             print(i*6+j, (i+1)*6+j,(i+2)*6+j, (i+3)*6+j)

            
            if board[i*6+j] == 0 and board[(i+1)*6+j] == player and board[(i+2)*6+j] == player and board[(i+3)*6+j] == player:
#                 print('e')
                return j
        
    # Check diagonal potential wins (left to right)
    for i in range(4):
        for j in range(3):
            
            if board[i*6+j] == 0 and board[(i+1)*6+j+1] == player and board[(i+2)*6+j+2] == player and board[(i+3)*6+j+3] == player:
#                 print('f')
                return j
            
    # Check diagonal potential wins (right to left)
    for i in range(4):
        for j in range(3, 6):
            if board[i*6+j] == 0 and board[(i+1)*6+j-1] == player and board[(i+2)*6+j-2] == player and board[(i+3)*6+j-3] == player:
#                 print('g')
                return j
        
    # If no potential wins, return random column
    k = []
#     print('k')
    for i in range(6):
        if board[i] == 0:
            k.append(i)
#     print(k)
    if len(k):
        return k[random.randint(0,len(k)-1)]
    else:
        return -1


In [12]:
def getValidRow(board, column):
    row = 6
    while row >= 0:
#         print(row * 6 + column)
        if board[row * 6 + column] == 0:
            break
        row -= 1
    if row < 0:
        return -1
    return row

In [13]:
def check_winner(board):
    rows = 7
    cols = 6
        
    for i in range(rows):
        for j in range(cols - 3):
            idx = i * cols + j
            if (board[idx] != 0 and
                board[idx] == board[idx+1] == board[idx+2] == board[idx+3]):
                return board[idx]

    for j in range(cols):
        for i in range(rows - 3):
            idx = i * cols + j
            if (board[idx] != 0 and
                board[idx] == board[idx+cols] == board[idx+2*cols] == board[idx+3*cols]):
                return board[idx]

    for i in range(rows - 3):
        for j in range(cols - 3):
            idx = i * cols + j
            if (board[idx] != 0 and
                board[idx] == board[idx+cols+1] == board[idx+2*cols+2] == board[idx+3*cols+3]):
                return board[idx]

    for i in range(3, rows):
        for j in range(cols - 3):
            idx = i * cols + j
            if (board[idx] != 0 and
                board[idx] == board[idx-cols+1] == board[idx-2*cols+2] == board[idx-3*cols+3]):
                return board[idx]
                
    return None


In [14]:
def run(j,itera):
    wins_computer = []
    wins_user = []
    ties = []
    
    for ii in range(itera):

        s = (0,0,0,0,0,0,
             0,0,0,0,0,0,
             0,0,0,0,0,0,
             0,0,0,0,0,0,
             0,0,0,0,0,0,
             0,0,0,0,0,0,
             0,0,0,0,0,0)
        

        i = j
        if i == 0:
            player = 1
            opponent = 2
        else:
            player = 2
            opponent = 1
            
        while not check_winner(s):
            if i%2==0:
#                 print('Random')
                move = check_potential_win(s, opponent)
#                 print(move)
                if move==-1:
                    ties.append(1)
                    break
                row = getValidRow(list(s),move)
                s = list(s)
                s[row*6+move] = player
            
            else:
#                 print('Computer')
                a, b = max_action(Q,s)
                s = list(s)
#                 game.rem(moves_array,a)
#                 if first_try:
#                     a = random.randint(0, 8)
#                     first_try = False            
                s[a] = opponent
                s = tuple(s)
#             printboard(s)
#             print(s)
            i = i + 1
                
                
        if check_winner(s)==opponent:
            wins_computer.append(1)
        elif check_winner(s)==player:
            wins_user.append(1)
        
#     print(check_winner(s))
    return len(wins_computer), len(wins_user), len(ties)


In [15]:
class Connect4:
    def __init__(self):
        self.player, self.opponent = 'o', 'x'
        
    def printboard(self, board):
        rows = len(board)
        cols = len(board[0])
        board = [[' ' if x == '_' else x for x in row] for row in board]
        for row in range(len(board)):
            print('|', end=' ')
            for col in range(len(board[row])):
                print('{} |'.format(board[row][col]), end=' ')
            print()
        print()

    
    def isMovesLeft(self, board):
        rows = len(board)
        cols = len(board[0])
        for i in range(rows):
            for j in range(cols):
                if board[i][j] == '_':
                    return True
        return False
    
    def check_winner(self, board):
        rows = len(board)
        cols = len(board[0])
        
        for i in range(rows):
            for j in range(cols - 3):
                if (board[i][j] != '_' and
                    board[i][j] == board[i][j+1] == board[i][j+2] == board[i][j+3]):
                    return board[i][j]

        for j in range(cols):
            for i in range(rows - 3):
                if (board[i][j] != '_' and
                    board[i][j] == board[i+1][j] == board[i+2][j] == board[i+3][j]):
                    return board[i][j]

        for i in range(rows - 3):
            for j in range(cols - 3):
                if (board[i][j] != '_' and
                    board[i][j] == board[i+1][j+1] == board[i+2][j+2] == board[i+3][j+3]):
                    return board[i][j]

        for i in range(3, rows):
            for j in range(cols - 3):
                if (board[i][j] != '_' and
                    board[i][j] == board[i-1][j+1] == board[i-2][j+2] == board[i-3][j+3]):
                    return board[i][j]
                
        return None
    
    
    def evaluate(self, board):
        
#         self.opponent = 'o' if self.player == 'x' else 'x'
        rows = len(board)
        cols = len(board[0])
        
        for i in range(rows):
            for j in range(cols - 3):
                if (board[i][j] != '_' and
                    board[i][j] == board[i][j+1] == board[i][j+2] == board[i][j+3]):
                    if board[i][j]==self.player:
                        return 10
                    elif board[i][j]==self.opponent :
                        return -10

        for j in range(cols):
            for i in range(rows - 3):
                if (board[i][j] != '_' and
                    board[i][j] == board[i+1][j] == board[i+2][j] == board[i+3][j]):
                    if board[i][j]==self.player:
                        return 10
                    elif board[i][j]==self.opponent :
                        return -10

        for i in range(rows - 3):
            for j in range(cols - 3):
                if (board[i][j] != '_' and
                    board[i][j] == board[i+1][j+1] == board[i+2][j+2] == board[i+3][j+3]):
                    if board[i][j]==self.player:
                        return 10
                    elif board[i][j]==self.opponent :
                        return -10
        
        for i in range(3, rows):
            for j in range(cols - 3):
                if (board[i][j] != '_' and
                    board[i][j] == board[i-1][j+1] == board[i-2][j+2] == board[i-3][j+3]):
                    if board[i][j]==self.player:
                        return 10
                    elif board[i][j]==self.opponent :
                        return -10
                
        return 0
    
    def getValidRow(self, board, column):
        row = 6
        while row >= 0:
            if board[row][column] == '_':
                break
            row -= 1
        if row < 0:
            return -1
        return row

    def minimax(self, board, depth, alpha, beta, isMax, isPrun):
        rows = len(board)
        cols = len(board[0])
        score = self.evaluate(board)

        if score == 10:
            return score

        if score == -10:
            return score

        if self.isMovesLeft(board) == False:
            return 0

        if depth == 0:
            return 0

        if isMax:
            best = -1000
            for j in range(cols):
                i = self.getValidRow(board, j)
                if i != -1:
                    board[i][j] = self.player
                    best = max(best, self.minimax(board, depth - 1, alpha, beta, not isMax, isPrun))
                    board[i][j] = '_'
                    if not isPrun:
                        alpha = max(alpha, best)
                        if beta <= alpha:
                            break
            return best

        else:
            best = 1000
            for j in range(cols):
                i = self.getValidRow(board, j)
                if i != -1:
                    board[i][j] = self.opponent
                    best = min(best, self.minimax(board, depth - 1, alpha, beta, not isMax, isPrun))
                    board[i][j] = '_'
                    if not isPrun:
                        beta = min(beta, best)
                        if beta <= alpha:
                            break
            return best

        
        
    def get_best_move(self, board, depth, isPrun):
        rows = len(board)
        cols = len(board[0])
        bestVal = -1000
        bestMove = (-1, -1)
        alpha = -1000
        beta = 1000
        for j in range(cols):
            i = self.getValidRow(board, j)
            if i != -1:
                board[i][j] = self.player
                moveVal = self.minimax(board, depth - 1, alpha, beta, False, isPrun)
                board[i][j] = '_'

                
                
                if (moveVal > bestVal) :
                    bestMove = (i, j)
                    bestVal = moveVal
                if not isPrun:
                    alpha = max(alpha, bestVal)
                    if beta <= alpha:
                        break

        return bestMove

In [16]:
game = Connect4()

In [55]:
Q = {}
alpha = 0.1
gamma = 0.8
epsilon = 0.4
Q = QAgent(alpha,gamma,epsilon,20000)

In [62]:
def run(j,depth,itera):
    wins_minmax = []
    wins_ql = []
    ties = []
    
    for zz in range(itera):
        board = [
            [ '_', '_', '_' ,'_', '_', '_'],
            [ '_', '_', '_' ,'_', '_', '_'],
            [ '_', '_', '_' ,'_', '_', '_'],
            [ '_', '_', '_' ,'_', '_', '_'],
            [ '_', '_', '_' ,'_', '_', '_'],
            [ '_', '_', '_' ,'_', '_', '_'],
            [ '_', '_', '_' ,'_', '_', '_'],
        ]
        s = (0,0,0,0,0,0,
             0,0,0,0,0,0,
             0,0,0,0,0,0,
             0,0,0,0,0,0,
             0,0,0,0,0,0,
             0,0,0,0,0,0,
             0,0,0,0,0,0)
        
        i = j
        isPrun = True

        player = 1
        opponent = 2

        if i==0:
            game.player = 'x'
            game.opponent = 'o'
        else:
            game.player = 'o'
            game.opponent = 'x'
            player = 2
            opponent = 1

        first_try = True
        while not game.check_winner(board):
            if i%2==0:
                print('Q-L')
                if first_try:
                    move = random.randint(0,5)
                    s = list(s)
                    row = game.getValidRow(board,move)
                    if row == -1:
                        ties.append(1)
                        break
                    s[6* row + move] = player
                    s = tuple(s)
                    board[row][move]  = game.player
                    first_try = False
                else:
                    a,b = max_action(Q,s)
                    s = list(s)
                    s[a] = player
                    s = tuple(s)
                    c = a%6
                    r = a//6
#                     print(a, r,c)
                    board[r][c] = game.player
                    


            else:
                print('MinMax')
                if first_try:
                    move = random.randint(0,5)
                    s = list(s)
                    row = game.getValidRow(board,move)
                    if row == -1:
                        ties.append(1)
                        break
                    s[6* row + move] = opponent
                    s = tuple(s)
                    board[row][move]  = game.opponent
                    first_try = False
                else:
                    a,b = game.get_best_move(board,depth,isPrun)
                    board[a][b] = game.opponent
                    s = list(s)
                    s[6* a + b] = opponent
                    s = tuple(s)
#                 print(a,b)
                


#             print('minmax_board')

#             game.printboard(board)
#             print('Q-L board')
            printboard(s)
        
            i = i + 1
#         print(game.check_winner(board))
        if game.check_winner(board)==game.player:
            wins_ql.append(1)
        elif game.check_winner(board)==game.opponent:
            wins_minmax.append(1)
        else:
            ties.append(1)
#         print(zz)
#         game.printboard(board)

#     print(len(wins_computer),len(wins_smart),len (ties))
    return len(wins_ql),len(wins_minmax),len (ties)
      

        

In [64]:
run(0,1,1)

Q-L
|   |   |   |   |   |   |
|   |   |   |   |   |   |
|   |   |   |   |   |   |
|   |   |   |   |   |   |
|   |   |   |   |   |   |
|   |   |   |   |   |   |
|   |   |   |   | x |   |


MinMax
|   |   |   |   |   |   |
|   |   |   |   |   |   |
|   |   |   |   |   |   |
|   |   |   |   |   |   |
|   |   |   |   |   |   |
|   |   |   |   |   |   |
| o |   |   |   | x |   |


Q-L
|   |   |   |   |   |   |
|   |   |   |   |   |   |
|   |   |   |   |   |   |
|   |   |   |   |   |   |
|   |   |   |   |   |   |
|   |   |   |   |   |   |
| o |   |   |   | x | x |


MinMax
|   |   |   |   |   |   |
|   |   |   |   |   |   |
|   |   |   |   |   |   |
|   |   |   |   |   |   |
|   |   |   |   |   |   |
| o |   |   |   |   |   |
| o |   |   |   | x | x |


Q-L
|   |   |   |   |   |   |
|   |   |   |   |   |   |
|   |   |   |   |   |   |
|   |   |   |   |   |   |
|   |   |   |   |   |   |
| o |   |   |   |   | x |
| o |   |   |   | x | x |


MinMax
|   |   |   |   |   |   |
|   |   |   |   |   |

(0, 1, 0)